In [13]:
import mlflow
import requests
import pandas as pd
import mlflow.pyfunc
from sklearn.preprocessing import StandardScaler
import joblib

In [14]:
df = pd.read_csv('/home/project/data/simulados/simulados.csv')
df = df.drop(['TIPO_PACIENTE',], axis=1)

In [15]:
scaler = joblib.load('/home/project/models/preprocess_utils/scaler.pkl')
encoder = joblib.load('/home/project/models/preprocess_utils/encoder.pkl')

In [16]:
encoded_columns  = encoder.transform(df[['SEXO', 'ESTATUS_CASO', 'RESULTADO_PCR']])
scaled_columns = scaler.fit_transform(df.drop(['SEXO', 'ESTATUS_CASO', 'RESULTADO_PCR'], axis=1))

In [17]:
encoded_scaled_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(['SEXO', 'ESTATUS_CASO', 'RESULTADO_PCR']))
scaled_df = pd.DataFrame(scaled_columns, columns=df.drop(['SEXO', 'ESTATUS_CASO', 'RESULTADO_PCR'], axis=1).columns)
data = pd.concat([encoded_scaled_df, scaled_df], axis=1)

In [3]:
mlflow.set_tracking_uri("https://dagshub.com/empleo-inducido/project-ml.mlflow")

model_name = "testing-model"
model_version = 1

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

2024/05/30 07:13:27 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - numpy (current: 1.22.0, required: numpy==1.26.4)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


In [43]:
# URL del servicio local
url = "http://localhost:8000/predict/"

# Enviar una solicitud POST al servidor
response = requests.post(url, json={"data": data.tolist()})

# Imprimir la respuesta
print(response.json())



{'prediction': [1, 1, 1, 1, 1]}
